In [1]:
import pandas as pd
import numpy as np

import fiona
import shapely.geometry

# https://github.com/cgcai/SVY21/tree/master/Python
import SVY21

In [2]:
# HDB Carpark information 
# https://data.gov.sg/dataset/hdb-carpark-information 

df = pd.read_csv("hdb-carpark-information.csv")
df['region'] = ""
df['latitude'] = ""
df['longitude'] = ""
df['regionID'] = ""
df.head()


,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,park_and_ride_scheme,region,latitude,longitude,regionID
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,NO,,,,
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,,,
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,,,
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,NO,,,,
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,NO,,,,


In [3]:
# Extract the five regions from the map
# https://data.gov.sg/dataset/master-plan-2014-region-boundary-no-sea

shape=[]
region=[]
with fiona.open("MP14/MP14_REGION_NO_SEA_PL.shp") as fiona_collection:
    for i in range(0,5):
        shape.append(shapely.geometry.asShape(fiona_collection[i]['geometry']))
        region.append(fiona_collection[i]['properties'][u'REGION_N'].split()[0])


In [4]:
region[0], region[1], region[2], region[3], region[4]


(u'CENTRAL', u'EAST', u'NORTH', u'NORTH-EAST', u'WEST')

In [6]:
# Initialize SVY21 to convert SVY21 coordinates to WGS84 latitude and longitude
cv = SVY21.SVY21()


# 0="Singapore", 1="North", 2="North-East", 3="East", 4="Central", 5="West".
translate = [4,3,1,2,5]

# For each HDB carpark location, locate which section of Singapore
for index, row in df.iterrows():
    point = shapely.geometry.Point(row['x_coord'],row['y_coord'])
    
    # convert SVY21 coordinates to longitude and latitude
    
    lat, lon = cv.computeLatLon(row['x_coord'],row['y_coord'])
    df.loc[index,'latitude']=lat
    df.loc[index,'longitude']=lon
    
    # determin which of 5 regions, the carpark belongs to
    
    for i in range(0,5):
        if point.within(shape[i]):
            df.loc[index,'region']=region[i]
            df.loc[index,'regionId']=translate[i]
            


In [7]:
df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,park_and_ride_scheme,region,latitude,longitude,regionID,regionId
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,NO,CENTRAL,1.29043,103.865,,4
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,CENTRAL,1.32157,103.884,,4
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,CENTRAL,1.28087,103.892,,4
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,NO,NORTH-EAST,1.27117,103.932,,2
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,NO,NORTH-EAST,1.2829,103.929,,2


In [8]:
df.groupby('region').size()

region
CENTRAL       604
EAST          314
NORTH         247
NORTH-EAST    426
WEST          439
dtype: int64

In [11]:
df[['address','latitude','longitude', 'regionId']].to_csv("location.csv", encoding='utf-8', index_col=True, header=False)